In [1]:
import requests
import json
import pandas as pd 
from pandas import *
import numpy as np
import itertools
from sklearn.decomposition import ProjectedGradientNMF
import numpy
import matplotlib.pyplot as plt
import numpy as np
hs = pd.read_csv('/Users/binxiang/Desktop/Insight/Week 2/SH_CB.csv')

#### Generate investor profile dataframe for sql

# make unique investor list
uinvestor = []
for irow_hs in range(len(hs.index)):
    uinvestor.extend(unique(hs.loc[irow_hs,'investor_company0':'investor_company11'].dropna().values.tolist() + hs.loc[irow_hs,'investor_user0':'investor_user23'].dropna().values.tolist() + hs.loc[irow_hs,'CB_Investor0':'CB_Investor7'].dropna().values.tolist()).tolist())
uinvestor = unique(uinvestor)
investor_sql = pd.DataFrame(index=range(len(uinvestor)),columns=['Name','Category','TimesInvest'])
# fill in empty investor dataframe
for i in range(len(uinvestor)):
    investor_sql.ix[i,'Name'] = uinvestor[i]
    investor_sql.ix[i,'TimesInvest'] = 0
for irow_hs in hs.index:
    inv = (unique(hs.loc[irow_hs,'investor_company0':'investor_company11'].dropna().values.tolist() + hs.loc[irow_hs,'investor_user0':'investor_user23'].dropna().values.tolist() + hs.loc[irow_hs,'CB_Investor0':'CB_Investor7'].dropna().values.tolist()).tolist())
    for i_inv in range(len(inv)):
        investor_sql['Category'][investor_sql['Name']==inv[i_inv]] = hs.loc[irow_hs,'CB_Category']
# fill in total times of investments from CrunchBase
for i_inv in range(len(uinvestor)):
    name = uinvestor[i_inv].replace(' ','-')
    name = name.replace('.','-')
    url='https://api.crunchbase.com/v/3/organizations/'+ name
    parameters = {'user_key': ['fcd09c06050fc1052f0aceb5b828d16f']}
    try:
        r = requests.get(url, params = parameters)
        test = json.loads(r.text)
        #n_company = len(test.get('data').get('relationships').get('investments').get('items'))
        n_invest = test.get('data').get('properties').get("number_of_investments")
        #investor_sql['TimesInvest'][investor_sql['Name']==uinvestor[i_inv]] = round(np.log10(n_invest) + n_invest/ float(n_company),1)
        investor_sql['TimesInvest'][investor_sql['Name']==uinvestor[i_inv]] = n_invest
    except:
        pass
for i_inv in range(len(uinvestor)):
    name = uinvestor[i_inv].replace(' ','-')
    name = name.replace('.','-')
    url='https://api.crunchbase.com/v/3/people/'+ name
    parameters = {'user_key': ['fcd09c06050fc1052f0aceb5b828d16f']}
    try:
        r = requests.get(url, params = parameters)
        test = json.loads(r.text)
        #n_company = len(test.get('data').get('relationships').get('investments').get('items'))
        n_invest = test.get('data').get('relationships').get('investments').get('paging').get('total_items')
        #investor_sql['TimesInvest'][investor_sql['Name']==uinvestor[i_inv]] = round(np.log10(n_invest) + n_invest / float(n_company),1)
        investor_sql['TimesInvest'][investor_sql['Name']==uinvestor[i_inv]] = n_invest
    except:
        pass
# save to sql
investor_sql.to_csv('investor_sql.csv')


### Generate company profile dataframe for sql

ucompany = unique(hs["COMPANY_NAME"])
company_sql = pd.DataFrame(index=range(len(ucompany)),columns=['Name','Category','Location','Title','Investors'])

# fill in empty company dataframe
for i in range(len(ucompany)):
    company_sql.ix[i,'Name'] = ucompany[i]
for irow_hs in hs.index:
    company_sql['Category'][company_sql['Name']==hs.loc[irow_hs,'COMPANY_NAME']] = hs.loc[irow_hs,'CB_Category']
    company_sql['Location'][company_sql['Name']==hs.loc[irow_hs,'COMPANY_NAME']] = hs.loc[irow_hs,'LOCATION']
    company_sql['Title'][company_sql['Name']==hs.loc[irow_hs,'COMPANY_NAME']] = hs.loc[irow_hs,'TITLE']
    inv = (unique(hs.loc[irow_hs,'investor_company0':'investor_company11'].dropna().values.tolist() + hs.loc[irow_hs,'investor_user0':'investor_user23'].dropna().values.tolist() + hs.loc[irow_hs,'CB_Investor0':'CB_Investor7'].dropna().values.tolist()).tolist())
    if len(inv) > 0:
        inv_str = inv[0]
        for i_inv in range(len(inv))[1:]:
            inv_str = inv_str + ' + ' + inv[i_inv]
        company_sql['Investors'][company_sql['Name']==hs.loc[irow_hs,'COMPANY_NAME']] = inv_str
# save to sql
company_sql.to_csv('company_sql.csv')


In [78]:
unique(company_sql['Title'])

array(['Seed', 'Debt', 'Series B', 'Series A', 'Series C', 'Grant',
       'Series D', 'Venture Round', 'Incubator', 'Acquired', 'Series F',
       'Series E', 'First Round Capital IV', 'Digital Growth Fund',
       'Correlation Ventures, LP', 'Founder', 'Series G',
       'CHCF Health Innovation Fund', 'West Health Investment Fund',
       'Welsh, Carson, Anderson & Stowe XI',
       'Summit Partners Venture Capital Fund II', 'Aberdare Ventures I LP',
       'Aisling Capital II LP', 'Bessemer Venture Partners Fund VII LP',
       'CHP I', 'Highland Consumer Fund I', 'Lux Ventures II',
       'Sequoia Israel Fourth Fund', 'Healthcare Innovation Fund',
       'Angel Fund', 'True Ventures III', 'Credo Stage 1',
       'Morgenthaler Ventures IX', 'NMP III', 'RHV', 'Debt Capital',
       'Fund II', 'North Bridge Growth Equity I', 'Fund I', 'Fund III',
       'Venrock V', 'Blue Cross Blue Shield Fund II', 'Apex VI',
       'New Enterprise Associates 13', 'Venture Capital Fund I',
       'Pf